# Basic Tutorial for TensorFlow


In [ ]:
!pip -q install aquvitae

In [ ]:
!wget -q --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1D28_TKauDVFO2nVlUeK_fuZX85zAO5o4' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1D28_TKauDVFO2nVlUeK_fuZX85zAO5o4" -O teacher.h5 && rm -rf /tmp/cookies.txt

In [ ]:
import tensorflow as tf

from aquvitae import dist, ST

In [ ]:
dataset = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = dataset.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

170508288/170498071 [==============================] - 13s 0us/step


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)

In [ ]:
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D,  MaxPool2D, Flatten, GlobalAveragePooling2D,  BatchNormalization, Layer, Add
from keras.models import Sequential
from keras.models import Model
import tensorflow as tf


class ResnetBlock(Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = BatchNormalization()
        self.conv_2 = Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = BatchNormalization()
        self.merge = Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out


class ResNet18(Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = Conv2D(64, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = BatchNormalization()
        self.pool_2 = MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(64)
        self.res_1_2 = ResnetBlock(64)
        self.res_2_1 = ResnetBlock(128, down_sample=True)
        self.res_2_2 = ResnetBlock(128)
        self.res_3_1 = ResnetBlock(256, down_sample=True)
        self.res_3_2 = ResnetBlock(256)
        self.res_4_1 = ResnetBlock(512, down_sample=True)
        self.res_4_2 = ResnetBlock(512)
        self.avg_pool = GlobalAveragePooling2D()
        self.flat = Flatten()
        self.fc = Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
            out = res_block(out)
        out = self.avg_pool(out)
        out = self.flat(out)
        out = self.fc(out)
        return out


In [ ]:
student = ResNet18(10)
student.build(input_shape = (None,32,32,3))

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
teacher = tf.keras.applications.ResNet50(weights=None, input_shape=(32, 32, 3), classes=10)
teacher.load_weights('teacher.h5')


In [ ]:
from tensorflow.keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
student.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 
student.summary()


Model: "res_net18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  9472      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 resnet_block (ResnetBlock)  multiple                  74368     
                                                                 
 resnet_block_1 (ResnetBlock  multiple                 74368     
 )                                                               
                                                         

In [ ]:
student.save('model.h5')

NotImplementedError: ignored

In [ ]:
model

In [ ]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [ ]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="frozen_graph.pb",
                  as_text=False)

--------------------------------------------------
Frozen model layers: 
x
resnet50/conv1_pad/Pad/paddings
resnet50/conv1_pad/Pad
resnet50/conv1_conv/Conv2D/ReadVariableOp/resource
resnet50/conv1_conv/Conv2D/ReadVariableOp
resnet50/conv1_conv/Conv2D
resnet50/conv1_conv/BiasAdd/ReadVariableOp/resource
resnet50/conv1_conv/BiasAdd/ReadVariableOp
resnet50/conv1_conv/BiasAdd
resnet50/conv1_bn/ReadVariableOp/resource
resnet50/conv1_bn/ReadVariableOp
resnet50/conv1_bn/ReadVariableOp_1/resource
resnet50/conv1_bn/ReadVariableOp_1
resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp/resource
resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp
resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp_1/resource
resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp_1
resnet50/conv1_bn/FusedBatchNormV3
resnet50/conv1_relu/Relu
resnet50/pool1_pad/Pad/paddings
resnet50/pool1_pad/Pad
resnet50/pool1_pool/MaxPool
resnet50/conv2_block1_0_conv/Conv2D/ReadVariableOp/resource
resnet50/conv2_block1_0_conv/Conv2D/ReadVariable

'./frozen_models/frozen_graph.pb'

In [ ]:
!git clone https://github.com/deepmodeling/deepmd-kit.git

Cloning into 'deepmd-kit'...
remote: Enumerating objects: 12957, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12957 (delta 0), reused 10 (delta 0), pack-reused 12947
Receiving objects: 100% (12957/12957), 14.31 MiB | 18.43 MiB/s, done.
Resolving deltas: 100% (9181/9181), done.


In [ ]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.io.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.compat.v1.GraphDef() 
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [ ]:
cd frozen_models/

/content/frozen_models


In [ ]:
graph = load_graph('frozen_graph.pb')

In [ ]:
for op in graph.get_operations():
        print(op.name)

prefix/x
prefix/resnet50/conv1_pad/Pad/paddings
prefix/resnet50/conv1_pad/Pad
prefix/resnet50/conv1_conv/Conv2D/ReadVariableOp/resource
prefix/resnet50/conv1_conv/Conv2D/ReadVariableOp
prefix/resnet50/conv1_conv/Conv2D
prefix/resnet50/conv1_conv/BiasAdd/ReadVariableOp/resource
prefix/resnet50/conv1_conv/BiasAdd/ReadVariableOp
prefix/resnet50/conv1_conv/BiasAdd
prefix/resnet50/conv1_bn/ReadVariableOp/resource
prefix/resnet50/conv1_bn/ReadVariableOp
prefix/resnet50/conv1_bn/ReadVariableOp_1/resource
prefix/resnet50/conv1_bn/ReadVariableOp_1
prefix/resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp/resource
prefix/resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp
prefix/resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp_1/resource
prefix/resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp_1
prefix/resnet50/conv1_bn/FusedBatchNormV3
prefix/resnet50/conv1_relu/Relu
prefix/resnet50/pool1_pad/Pad/paddings
prefix/resnet50/pool1_pad/Pad
prefix/resnet50/pool1_pool/MaxPool
prefix/resnet50/conv2_block

In [ ]:
with tf.io.gfile.GFile("frozen_graph.pb", "rb") as f:
  graph_def = tf.compat.v1.GraphDef()
  loaded = graph_def.ParseFromString(f.read())

    # Wrap frozen graph to ConcreteFunctions
frozen_func = wrap_frozen_graph(graph_def=graph_def,
                                    inputs=["x:0"],
                                    outputs=["Identity:0"],
                                    print_graph=True)

--------------------------------------------------
Frozen model layers: 
x
resnet50/conv1_pad/Pad/paddings
resnet50/conv1_pad/Pad
resnet50/conv1_conv/Conv2D/ReadVariableOp/resource
resnet50/conv1_conv/Conv2D/ReadVariableOp
resnet50/conv1_conv/Conv2D
resnet50/conv1_conv/BiasAdd/ReadVariableOp/resource
resnet50/conv1_conv/BiasAdd/ReadVariableOp
resnet50/conv1_conv/BiasAdd
resnet50/conv1_bn/ReadVariableOp/resource
resnet50/conv1_bn/ReadVariableOp
resnet50/conv1_bn/ReadVariableOp_1/resource
resnet50/conv1_bn/ReadVariableOp_1
resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp/resource
resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp
resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp_1/resource
resnet50/conv1_bn/FusedBatchNormV3/ReadVariableOp_1
resnet50/conv1_bn/FusedBatchNormV3
resnet50/conv1_relu/Relu
resnet50/pool1_pad/Pad/paddings
resnet50/pool1_pad/Pad
resnet50/pool1_pool/MaxPool
resnet50/conv2_block1_0_conv/Conv2D/ReadVariableOp/resource
resnet50/conv2_block1_0_conv/Conv2D/ReadVariable

In [ ]:
frozen_graph_predictions = frozen_func(x=tf.constant(X))[0]

In [ ]:
print(frozen_graph_predictions[0].numpy())

[1.7223044e-06 5.3123972e-06 1.2021983e-06 2.2668419e-03 4.6424498e-06
 9.9734592e-01 4.3024977e-07 3.2340703e-04 1.5316878e-05 3.5045006e-05]


In [ ]:
frozen_graph_predictions

<tf.Tensor: shape=(10000, 10), dtype=float32, numpy=
array([[1.7223044e-06, 5.3123972e-06, 1.2021983e-06, ..., 3.2340703e-04,
        1.5316878e-05, 3.5045006e-05],
       [9.5533663e-03, 9.5753716e-03, 7.0554876e-08, ..., 4.0406057e-08,
        9.8084164e-01, 2.9317882e-05],
       [1.7875287e-03, 7.4147480e-04, 1.1522280e-06, ..., 1.8718022e-07,
        9.9742436e-01, 4.4671982e-05],
       ...,
       [6.2900933e-09, 9.3190211e-09, 6.7394758e-05, ..., 1.0796351e-07,
        3.8997293e-08, 3.1504388e-08],
       [8.6507236e-05, 9.9864131e-01, 1.0470835e-05, ..., 3.0635858e-06,
        6.5371833e-06, 2.0140754e-04],
       [6.0298760e-08, 2.5708007e-07, 8.9584063e-07, ..., 9.9966860e-01,
        1.9929644e-08, 1.8595736e-06]], dtype=float32)>

In [ ]:
X = tf.cast(x_test, tf.float32)

In [ ]:
def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

    if print_graph == True:
        print("-" * 50)
        print("Frozen model layers: ")
        layers = [op.name for op in import_graph.get_operations()]
        for layer in layers:
            print(layer)
        print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

In [ ]:
alpha = dist(
    teacher=frozen_func,
    student=frozen_func,
    algo=ST(alpha=0.6, T=2.5),
    optimizer=tf.keras.optimizers.Adam(),
    train_ds=train_ds,
    test_ds=test_ds,
    iterations=300
)

NotImplementedError: ignored

In [ ]:
student = dist(
    teacher=teacher,
    student=student,
    algo=ST(alpha=0.6, T=2.5),
    optimizer=tf.keras.optimizers.Adam(),
    train_ds=train_ds,
    test_ds=test_ds,
    iterations=105000
)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

In [ ]:
student = dist(
    teacher=teacher,
    student=teacher,
    algo=ST(alpha=0.6, T=2.5),
    optimizer=tf.keras.optimizers.Adam(),
    train_ds=train_ds,
    test_ds=test_ds,
    iterations=300
)

Training - 300/300 [██████████████████████████████] ELP: 01:22, accuracy: 0.9443 - val_accuracy: 0.7065 


In [ ]:
student = dist(
    teacher=teacher,
    student=teacher,
    algo=ST(alpha=0.6, T=2.5),
    optimizer=tf.keras.optimizers.Adam(),
    train_ds=train_ds,
    test_ds=test_ds,
    iterations=100000
)

Training - 463/100000 [▏                             ] ELP: 00:52, accuracy: 0.9274 - val_accuracy: 0.7144 

KeyboardInterrupt: ignored